## **Turi Create Object Detection on Skafos**
The below example code and more documentation on the approach can be found [here](https://apple.github.io/turicreate/docs/userguide/activity_classifier).
This notebook does the following:
1. Loads the data
2. Builds the model
3. Saves the model

In [1]:
import turicreate as tc
from skafossdk import *
import sys
import pandas as pd
from common.load_data import ActivityData
import common.save_models as sm


ska = Skafos() # initialize Skafos

In [4]:
tc.config.set_num_gpus(0)

### 1. **Load the data**
The data for this example comes from the open source HAPT dataset. The data is loaded in a separate module `common.load_data`. The data is then split into training and testing sets.

In [2]:
# load data loading class
activity_data = ActivityData()
data = activity_data.get_data() 


train, test = tc.activity_classifier.util.random_split_by_session(data, session_id='Experiment', fraction=0.8)

### 2. **Build the model**
We pass the data to the `tc.activity_classifier.create` function and specify a few arguments needed to properly run the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/activity_classifier/).

In [5]:
# Create an activity classifier
model = tc.activity_classifier.create(train, session_id='Experiment', target='Activity', prediction_window=50)

The dataset has less than the minimum of 100 sessions required for train-validation split. Continuing without validation set


Pre-processing 600945 samples...

Using sequences of size 1000 for model creation.

Processed a total of 49 sessions.

Using CPU to create model
+----------------+----------------+----------------+----------------+
| Iteration      | Train Accuracy | Train Loss     | Elapsed Time   |
+----------------+----------------+----------------+----------------+
| 1              | 0.613          | 0.969          | 1.0            |
| 2              | 0.798          | 0.538          | 2.0            |
| 3              | 0.859          | 0.391          | 3.0            |
| 4              | 0.877          | 0.333          | 4.0            |
| 5              | 0.887          | 0.309          | 5.0            |
| 6              | 0.896          | 0.276          | 6.1            |
| 7              | 0.901          | 0.254          | 7.1            |
| 8              | 0.900          | 0.250          | 8.1            |
| 9              | 0.909          | 0.231          | 9.1            |
| 10             | 0.917          | 0.212          | 10.1           |
+----------------+----------------+----------------+------------

### 3. **Save the model**
We save the model to Skafos so that you can download it into your project later without having to always retrain the model.

In [ ]:
# export to coreml
coreml_model_name = "activity_classifier.mlmodel"
res = model.export_coreml(coreml_model_name)

# compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

# save to Skafos
sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
                         compressed_model = compressed_model,
                         permissions = 'public')
